# Naive Byes Text Classification

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def count(word,list):
    c=0
    for i in list:
        if(i==word):
            c+=1
    return c

In [ ]:
def remove_stop_words(text=[]):
    stop_words=["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "you're", "you've", "you'll", "you'd", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "she's", "her", "hers", "herself", "it", "it's", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "that'll", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "don't", "should", "should've", "now", "d", "ll", "m", "o", "re", "ve", "y", "ain", "aren", "aren't", "couldn", "couldn't", "didn", "didn't", "doesn", "doesn't", "hadn", "hadn't", "hasn", "hasn't", "haven", "haven't", "isn", "isn't", "ma", "mightn", "mightn't", "mustn", "mustn't", "needn", "needn't", "shan", "shan't", "shouldn", "shouldn't", "wasn", "wasn't", "weren", "weren't", "won", "won't", "wouldn", "wouldn't"]
    word=""
    text+=""
    l=len(text)
    clean_text_list=[]
    for i in range(l):
        if(text[i]==" " or i==(l-1)):
            if(i==(l-1)):
                word+=text[i]
            f=count(word.lower(),stop_words)
            if(f==0):
                clean_text_list.append(word)
            word=""
            continue
        word+=text[i]
    return clean_text_list

In [ ]:
def generate_features(email):
    email=np.array(email)
    feature_list=[]
    frequency_list=[]
    l=len(email)
    for i in range(l):
        text=email[i]
        text=str(text)
        word_list=remove_stop_words(text)
        words,frequency=np.unique(word_list,return_counts=True)
        l1=len(frequency)
        for j in range(l1):
            if(frequency[j]>6):
                feature_list.append(words[j])
    feature_list=list(dict.fromkeys(feature_list))
    return feature_list

In [ ]:
def generate_data(email,feature_names):
    email=np.array(email)
    data=[]
    l=len(email)
    for i in range(l):
        d=[]
        text=email[i]
        text=str(text)
        word_list=remove_stop_words(text)
        for feature in feature_names:
            frequency=count(feature,word_list)
            d.append(frequency)
        data.append(d)
    return data

In [ ]:
def generate_dictionary(data,y_train):
    result={}
    classes=set(y_train)
  #  data=np.array(data)
    y_train=np.array(y_train)
    for current_class in classes:
        result[current_class]={}
        result["total_data"]=len(y_train)
        current_class_rows=(y_train==current_class)
        x_train_current=data.iloc[current_class_rows,:]
        y_train_current=y_train[current_class_rows]
        l=data.shape[1]-1
        features=data.columns
        for i in range(l):
            result[current_class][features[i].lower()]=x_train_current.iloc[:,i].sum()
        result[current_class]["total_count"]=len(y_train_current)
    return result

In [ ]:
def fit(x_train,y_train):
    key_words=generate_features(x_train)
    key_word_frequency=generate_data(x_train,key_words)
    data=pd.DataFrame(key_word_frequency,columns=key_words)
    dictionary=generate_dictionary(data,y_train)
    return dictionary

In [ ]:
def probability(current_class,x,dictionary={}):
    x=str(x)
    output=np.log(dictionary[current_class]["total_count"])-np.log(dictionary["total_data"])
    word_list=remove_stop_words(x)
    features=dictionary[current_class].keys()
    for word in word_list:
        word=word.lower()
        word_count=count(word,features)
        if(word_count>0):
            l=len(features)
            #for i in range(l):
         #       if(features[i]==word):
          #          break
            count_current_class_with_word=dictionary[current_class][word]+1
            count_current_class=dictionary[current_class]["total_count"]+len(dictionary[current_class].keys())-1
            current_probaility=word_count*(np.log(count_current_class_with_word)-np.log(count_current_class))
            output=output+current_probaility
    return output

In [ ]:
def single_point_predict(x,dictionary):
    x=np.array(x)
    classes=dictionary.keys()
    best_c=-1
    best_p=-1
    first_run=True
    for current_class in classes:
        if(current_class=="total_data"):
            continue
        current_prob=probability(current_class,x,dictionary)
        if(first_run==True or best_p<current_prob):
            best_p=current_prob
            best_c=current_class
        first_run=False
    return best_c

In [ ]:
def predict(x_test,dictionary):
    x_test=np.array(x_test)
    y_pred=[]
    for x in x_test:
        pred=single_point_predict(x,dictionary)
        y_pred.append(pred)
    return y_pred

In [ ]:
from sklearn.model_selection import train_test_split
spam=pd.read_csv(r'C:\Users\Armaan\Desktop\WORKSPACE\Datasets\Spam_Not_Spam_Dataset\spam_or_not_spam.csv')
x_train,x_test,y_train,y_test=train_test_split(spam.email,spam.label)

In [ ]:
dictionary=fit(x_train,y_train)         

In [ ]:
y_test_pred=predict(x_test,dictionary)
#y_train_pred=predict(x_train,dictionary)

In [ ]:
print(y_test_pred)

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(y_test,y_test_pred))
print(confusion_matrix(y_test,y_test_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.98       633
           1       0.99      0.74      0.84       117

    accuracy                           0.96       750
   macro avg       0.97      0.87      0.91       750
weighted avg       0.96      0.96      0.95       750

[[632   1]
 [ 31  86]]
